In [10]:
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium import webdriver

from time import sleep
from tqdm import tqdm_notebook as tqdm

In [2]:
import requests
from bs4 import BeautifulSoup as bs

In [90]:
get_inners = '''

hashCode = function(s){
  return s.split("").reduce(function(a,b){a=((a<<5)-a)+b.charCodeAt(0);return a&a},0);              
}

probview = document.getElementsByClassName('prob_view');

inners = [];

for (i = 0; i < probview.length; i++) {
  try {
  task_id = probview[i].getElementsByTagName('a')[0].innerText;
  
  data = probview[i].firstChild.firstChild.firstChild.nextElementSibling;
  
  data_inner = data.innerHTML;
  data_inner = data_inner.split(RegExp(' width=".*?"')).join('');
  data_inner = data_inner.split(RegExp(' height=".*?"')).join('');
  data_inner = data_inner.split(RegExp(' style=".*?"')).join('');
  data_inner = data_inner.split(RegExp(' class=".*?"')).join('');
  data_inner = data_inner.split(RegExp(' align=".*?"')).join('');
  
  hash = hashCode(data_inner);
  
  inners.push([hash, task_id]);
  }
  catch (e) {
       console.log(e)
    }
}
return inners
'''

In [3]:
def write_html(name,content):
    open(f'{name}.html','wb').write(content)

In [4]:
def get_task_innerHTML(task):
    pattern = ' class="(.*?)"'
    
    task_content = task.find('div').find('div').find_all('div')[1]
    innerHTML = task_content.encode_contents().decode('utf-8')
    return re.sub(pattern, '', innerHTML).replace('/>','>').replace('\xa0','&nbsp;'), task.find('a')['href']

In [62]:
def get_tasks_from_theme(theme_url):
    skip = 0
    result = []
    while True:
        res = requests.post(theme_url,data = {'ajax':1,'skip':skip})
        soup = bs(res.content, 'lxml')
        tasks = soup.find_all('div', {'class': 'prob_view'})
        if len(tasks) == 0:
            break
        result.append(res.content)
        skip += 5
    
    concated = ''
    for task_block in result:
        concated += task_block.decode('utf-8')
    
    return concated

In [ ]:
def update_page_source(task_block):
    current_html = driver.find_element_by_tag_name("html")
    driver.execute_script("arguments[0].innerHTML = arguments[1]", current_html, task_block)


In [11]:
driver = webdriver.Chrome()

In [52]:
theme_links = [a for a, b in [i.split('|||') for i in open('themes_list.txt','r').read().split('\n')] if b !='0']

In [ ]:
inners_dataset = []
for url in tqdm(theme_links):
    tasks_concated = get_tasks_from_theme(url)
    update_page_source(tasks_concated)
    sleep(0.1)
    inners = driver.execute_script(get_inners)
    inners_dataset += inners

A Jupyter Widget

In [84]:
len(inners_dataset)

10012

In [ ]:
driver.quit()

In [82]:
a = get_tasks_from_theme(theme_links[331])

In [83]:
update_page_source(a)

In [99]:
len(list(set([a for a, b in inners_dataset])))

33689

In [91]:
driver.execute_script(get_inners)

[[-630495975, '9985'],
 [-2044552141, '10264'],
 [-1422063601, '10265'],
 [-1771832858, '10266'],
 [1532597076, '10267'],
 [-215930294, '10268'],
 [-162988918, '10269'],
 [-1381197821, '10270'],
 [2034535759, '10271'],
 [584568807, '10272'],
 [-49030824, '10273'],
 [576690568, '10274'],
 [-653005661, '10352'],
 [1609322978, '10377'],
 [1010305485, '10402'],
 [224712192, '10427'],
 [912954708, '10645'],
 [1920706632, '10670'],
 [-2141554325, '10764'],
 [1729935538, '10789'],
 [-1518069058, '10822'],
 [-1768540758, '10874'],
 [430625834, '10899'],
 [2138143308, '10952'],
 [-1121017512, '12349'],
 [-1407383417, '12376'],
 [-1065106545, '12401'],
 [-997256472, '12426'],
 [1120634187, '12453'],
 [-1982484611, '12478'],
 [627356095, '12503'],
 [-130597920, '12529'],
 [-1641164705, '12554'],
 [193637728, '12579'],
 [-85950404, '12604'],
 [686444886, '12629'],
 [-1625369558, '12758'],
 [1454455631, '12784'],
 [1237500023, '12816'],
 [191699518, '12842'],
 [2036956628, '13724'],
 [1952730169, '

In [ ]:
import json
with open('all.json', 'w') as fp:
    json.dump(inners_dataset, fp)

In [24]:
with open('hash_id_dict.json', 'r') as f:
    distros_dict = json.load(f)

In [30]:
open('dict_in_txt.txt', 'w').write(str(distros_dict))

828403